In [385]:
include("SchwarzChristoffel.jl")

SchwarzChristoffel

In [386]:
p = SchwarzChristoffel.Polygon([-1.0,1.0,0.5,-1.0],[-1.0,-1.0,1.0,1.0])

Polygon with 4 vertices at Complex{Float64}[-1.0-1.0im, 1.0-1.0im, 0.5+1.0im, -1.0+1.0im] 
             interior angles/π = [0.5, 0.422, 0.578, 0.5]


In [ ]:
SchwarzChristoffel.Polygons.plot(p)

In [ ]:
z = [0.71+im*0.2 0.2+im*0]
SchwarzChristoffel.Polygons.plot(p)
PyPlot.plot(real.(z),imag.(z),marker="o")
SchwarzChristoffel.Polygons.isinpoly.(z,p,1e-5)

In [336]:
z1 = -1.0+0.0im #1.0im
z2 = exp(im*5π/4) #exp(im*3π/4)
sing1 = 2 #1
beta = [0.5,0.5,0.5,0.5]
z = [1.0im,-1.0,-1.0im,1.0]
qdat = SchwarzChristoffel.Integration.qdata(beta,8)
dabsquad2 = SchwarzChristoffel.Exterior.DabsQuad(z,beta,qdat)
I = dabsquad2(z1,z2,sing1)

0.8472130847939816

In [337]:
z1 = [-1.0+0.0im,0.0+1.0im]
z2 = [exp(im*5π/4),exp(im*3π/4)]
sing1 = [2,1]
I = dabsquad2.(z1,z2,sing1)

2-element Array{Float64,1}:
 0.847213
 0.847213

In [338]:
function deparam(w::Vector{Complex128},beta::Vector{Float64},
                 zeta0::Vector{Complex128},qdat)
  # w clockwise
  # beta turning angles

  n = length(w)
  if n == 2
    zeta = [-1,1]
  else
    len = abs.(diff(circshift(w,1)))
    nmlen = abs.(len[3:n-1]/len[2])
    if isempty(zeta0)
      y0 = zeros(n-1)
    else
      zeta0 = zeta0/zeta0[n]
      θ = angle.(zeta0)
      θ[θ.<=0] = θ[θ.<=0] + 2π
      dt = diff([0;θ[1:n-1];2π])
      @. y0 = log(dt[1:n-1]/dt[2:n])
    end

    depfun!(F,y) = SchwarzChristoffel.Exterior.depfunfull!(F,y,n,beta,nmlen,qdat)

    F0 = similar(y0)
    df = OnceDifferentiable(depfun!, y0, F0)
    sol = nlsolve(depfun!,y0,show_trace = :true)

    zeta, θ = y_to_zeta(sol.zero)

  end

  mid = zeta[1]*exp(0.5*im*angle(zeta[2]/zeta[1]))
  dabsquad = SchwarzChristoffel.Exterior.DabsQuad(zeta,beta,qdat)
  c = (w[2]-w[1])/(dabsquad(zeta[1],mid,1)-dabsquad(zeta[2],mid,2))

  return zeta, c

end
function y_to_zeta(y::Vector{Float64})

    cs = cumsum(cumprod([1;exp.(-y)]))
    n = length(cs)
    θ = 2π*cs[1:n-1]./cs[n]
    zeta = ones(Complex128,n)
    zeta[1:n-1] = exp.(im*θ)
    return zeta, θ

  end

y_to_zeta (generic function with 1 method)

In [378]:
function depfunfull!(F,y,n,beta,nmlen,qdat)  
    
  zeta, θ = SchwarzChristoffel.Exterior.y_to_zeta(y)  
  mid = exp.(im*0.5*(θ[1:n-2]+θ[2:n-1]))
  #dabsquad(z1,z2,sing1) = SchwarzChristoffel.Exterior.dabsquad(z1,z2,sing1,zeta,beta,qdat)
  dabsquad = SchwarzChristoffel.Exterior.DabsQuad(zeta,beta,qdat)
  ints = dabsquad.(zeta[1:n-2],mid,collect(1:n-2)) + dabsquad.(zeta[2:n-1],mid,collect(2:n-1))
    println(ints)
    
  if n > 3
    F[1:n-3] = abs.(ints[2:n-2])/abs(ints[1]) - nmlen
  end

  res = -sum(beta./zeta)/ints[1]
  F[n-2] = real(res)
  F[n-1] = imag(res)  
    
end

depfunfull! (generic function with 2 methods)

In [379]:
w = flipdim(p.vert,1)
beta = 1.0-flipdim(p.angle,1)
len = abs.(diff(circshift(w,1)))
nmlen = abs.(len[3:n-1]/len[2])
F = zeros(n-1)
y0 = zeros(n-1)
depfunfull!(F,y0,n,beta,nmlen,qdat)
println(F)
SchwarzChristoffel.Exterior.depfunfull!(F,y0,n,beta,nmlen,qdat)
println(F)

0.8896074270553815
0.917211037029498
0.9517649480908896
0.7857931143315692
[1.84137, 1.703]
[-0.449513, -0.0423484, -0.0423484]
0.8896074270553815
0.917211037029498
0.9517649480908896
0.7857931143315692
[1.84137, 1.703]
[-0.449513, -0.0423484, -0.0423484]


In [339]:
w = flipdim(p.vert,1)
beta = 1.0-flipdim(p.angle,1)
zeta0 = Complex128[]
n = length(w)
tol = 1e-8
nqpts = max(ceil(Int,-log10(tol)),2)
qdat = SchwarzChristoffel.Integration.qdata(beta,nqpts)
#len = abs.(diff(circshift(w,1)))
#nmlen = abs.(len[3:n-1]/len[2])
#depfun!(F,y) = depfunfull!(F,y,n,beta,nmlen,qdat)

#y0 = zeros(n-1)
#F0 = similar(y0)
#df = OnceDifferentiable(depfun!, y0, F0)
#sol = nlsolve(depfun!,y0,show_trace = :true)

#zeta, θ = SchwarzChristoffel.Exterior.y_to_zeta(sol.zero)
#zeta
zeta, c = deparam(w,beta,zeta0,qdat)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     3.433396e-01              NaN
     1     6.010255e-02     2.406970e-01
     2     2.486045e-03     3.495361e-02
     3     1.732264e-06     2.920999e-03
     4     2.300974e-12     2.015030e-06


(Complex{Float64}[-0.0195313+0.999809im, -0.984827+0.173541im, -0.1291-0.991632im, 1.0+0.0im], -29.350650310183017 - 0.0im)

In [331]:
zeta

4-element Array{Complex{Float64},1}:
 -0.0195313+0.999809im
  -0.984827+0.173541im
    -0.1291-0.991632im
        1.0+0.0im     

In [ ]:
SchwarzChristoffel.Exterior.Extermap(p)

In [ ]:
deparam(flipdim(p.vert,1),1.-flipdim(p.angle,1),Complex128[],qdat)

In [387]:
zeta,c = SchwarzChristoffel.Exterior.deparam(w,beta,zeta0,qdat)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     4.495126e-01              NaN
     1     1.252776e-01     3.431283e-01
     2     5.028244e-03     5.025959e-02
     3     9.186100e-06     2.103350e-03
     4     3.262679e-11     5.951120e-06
-0.6618235122093528 + 0.7496596819136512im
Complex{Float64}[-1.0+1.0im, 0.5+1.0im]


(Complex{Float64}[-0.0567902+0.998386im, -0.983643+0.180131im, -0.0977323-0.995213im, 1.0+0.0im], -30.681522225215488 - 0.0im)

In [384]:
c

-30.681522225215488 - 0.0im

In [90]:
θ = angle.(p.vert)
θ[θ.<=0] = θ[θ.<=0] + 2π
diff([0;θ[1:end-1];2π])

4-element Array{Float64,1}:
  3.92699
  1.5708 
 -4.39064
  5.17604

In [37]:
function arraystuff2(a::Vector{Float64},b::Vector{Complex128})
    @assert length(a)==length(b)
    a[1]
end

arraystuff2 (generic function with 1 method)

In [41]:
arraystuff2([1.0;2.0],Complex128[1;4;5])

LoadError: [91mAssertionError: length(a) == length(b)[39m

In [45]:
typeof(Complex128[])

Array{Complex{Float64},1}

In [53]:
typeof(max(ceil(Int,-log10(1e-8)),2))

Int64

In [49]:
sizeof(Integer)

LoadError: [91margument is an abstract type; size is indeterminate[39m